# TOTEMS - Tidal Orbital decay Timing Extrapolation & Modelling Software

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

#Import curve fitting (Which is what we need for tidal decay)
from scipy.optimize import curve_fit

#Import pylightcurve, used for BJD HJD conversions - thanks to Angelos Tsiaras
import pylightcurve as plc

In [ ]:
%matplotlib notebook
# if this isn't in a separate cell, sometimes doesn't work right
# magic commands are weird, to say the least.

# TODO - use structured or recorded arrays with boolean masks, rather than repeatedly iterating

First function: gets the relevant data for the exoplanet from the [http://var2.astro.cz/ETD/](Exoplanet Transit
Database). Use this if you don't
have any archive data of your own to use, though really any data from a recent paper will probably be better.

In [ ]:
def get_etd_data(right_ascension, declination, url):
    """Imports the archive data from ETD. Uses the RA & Dec of a target to convert HJD_UTC to BJD_TDB.
    Inputs:
    - ra, the RA in hh mm ss.ss
    - dec, the Declination in hh mm ss.ss
    - url, the URL of datafile
    Outputs:
    - no, the number of the entry in ETD's list
    - bjd, t_mid converted to BJD_TDB
    - mid_err, the uncertainty in t_mid
    - epoch, the epoch number for the transit based on the t_0 and period on ETD.
    - dq, ETD's Data Quality measurement. 1 is best, 5 is worst.
    """
    # Get the period and mid-time @ epoch 0 automatically from ETD
    p_0, t_0 = p0_t0_from_etd(url)

    # Now for the actual transit data: use genfromtxt as there may be missing values.
    # Import the data into a series of arrays using numpy loadtxt. Note we skip the first 5 rows and only use
    # certain columns, because there's lots of extra stuff we aren't using
    no,hjd,mid_err,epoch,dq = np.genfromtxt(url, encoding='unicode_escape', delimiter=';', skip_header=5, missing_values='', filling_values=-1, usecols=(0,1,2,3,10), unpack=True)

    # convert from the truncated HJD to the full HJD
    hjd = hjd + 2400000

    # assuming ETD uses HJD_UTC, convert to BJD_TDB.
    # of course, it might be worth checking that each individual data point specified is ACTUALLY HJD,
    # as I don't think ETD vets for those kinds of mistakes

    ra_dec_string = right_ascension+" "+declination #concatenate into one string, for fn converting to degrees
    right_ascension, declination = plc.ra_dec_string_to_deg(ra_dec_string) # convert from hh mm ss.ss to degrees

    # Convert to BJD_TDB. N.B. we assume uncertainty remains the same.
    bjd = [] # make a blank array to store BJD in
    for hjd_to_convert in hjd:
        bjd.append(plc.hjd_utc_to_bjd_tdb(ra, dec, hjd_to_convert)) # do the actual converting

    # return the data arrays. Notice this means you end up with five arrays, where each of the five elements is an
    # array for number, BJD, uncertainty, epoch, DQ. Also, two constants, the period and time at epoch 0.
    # Yes, this is a dreadful situation to be in. A high priority todo is to rewrite these into "datapoint" objects
    # where each object stores a mid-transit time and uncertainty, its number, epoch, and DQ.
    return no, bjd, mid_err,epoch,dq, p_0, t_0

In [ ]:
def p0_t0_from_etd(url):
    """Uses ETD to get values of t_0 and p_0, the mid-time and period at epoch 0.
    Inputs:
    -url, the url of the ETD data in ASCII format
    Outputs:
    -t_0, the mid-time @ epoch=0
    -p_0, the period @ epoch=0"""
    p_str, t_str = np.loadtxt(url, dtype=str, encoding='unicode_escape', delimiter=', ', skiprows=2, max_rows=1)

    for possibility in p_str.split():
        try:
            str(float(possibility))
            p_0 = possibility
        except ValueError:
            pass # if its not the number, move on

    for possibility in t_str.split():
        try:
            str(float(possibility))
            t_0 = possibility
        except ValueError:
            pass # if its not the number, move on

    return p_0, t_0

In [ ]:
def filter_etd_data(no, bjd, mid_err, epoch, dq, filter_dq):
    """Filters ETD data. Checks uncertainty exists. Returns only datapoints with DQ equal or better than specified
    value. N.B. DQ 1 is best, 5 is worst.
    Inputs:
    - no
    - bjd
    - mid_err
    - epoch
    - dq
    - filter_dq
    Outputs:
    - filteredData, the filtered data array (of arrays). Contains three arrays: BJD mid-time, uncertainty, and epoch.
    """

    # Let's filter for "good" data. I don't trust data without a tmid uncertainty, so check it exists/isn't 0
    good_data=[ [],[],[],[],[] ]
    for i,err in enumerate(mid_err):
        if err < 0:
            good_data[0].append(no[i])
            good_data[1].append(bjd[i])
            good_data[2].append(mid_err[i])
            good_data[3].append(epoch[i])
            good_data[4].append(dq[i])

    # the plan: one array, contains three arrays - these three arrays are for mid, mid_err, epoch
    # we don't bother preserving no., it was just imported to be used basically as a debugging tool
    # and dq is pointless once we've filtered. This also makes the addition of non-ETD data far easier
    filtered_data = [[],[],[]]

    for i,good_dq in enumerate(good_data[4]): # for each entry in the data
        if good_dq <= filter_dq: # check the DQ vs the specified DQ argument. If better...

            # ...then add the data to each of the three arrays in filteredData that we care about
            filtered_data[0].append(good_data[1][i])
            filtered_data[1].append(good_data[2][i])
            filtered_data[2].append(good_data[3][i])

    return filtered_data # return the data array containing only the transits that's been filtered by DQ

## Statistics functions

These functions are for the $\chi^2$ comparison. I sincerely doubt that I haven't accidentally reinvented the wheel - functions for this probably already exist. However, it's so simple, I've not exactly wasted hours on these.

Equations are from "Measurements and Their Uncertainties: A Practical Guide to Modern Error Analysis: Hughes and
Hase 2010".

$$ \chi^2 = \sum_i{\frac{y_i-y(x_i)}{\alpha_i^2}}$$

$\nu$ is the degrees of freedom: the number of datapoints minus the number of fitted parameters. We divide $\chi^2$ by
 $\nu$ to obtain the reduced chi-squared, as follows:

$$ \chi^2_\text{reduced} = \frac{\chi^2}{\nu} $$

In [ ]:
def chi_squared(y_i,yx,alpha):
    """Calculates the unreduced chi squared.
    Inputs:
    - y_i, the array of observed y value (the y_i in the formula)
    - yx, the array of y values of the fitted line (the y(x) in the formula )
    - alpha, array of error in y, in the same units as y
    Outputs:
    - chi2, the unreduced chi squared value
    """

    chi2=0
    for i,y in enumerate(y_i):
        chi2 += ((y-yx[i])**2) / (alpha[i]**2)
    return chi2

def reduced_chi_squared(y,yx,alpha,m):
    """Calculates the reduced chi squared from the raw data: just chisq divided by degrees of freedom
    Degree of freedom is just number of observations n - number of fitted parameters m
    where n is just the number of y (or y(x) or x) values
    Inputs:
    - y, array of actual observed y value (the y_i in the formula)
    - yx, array of y value of the fitted line (the y(x) in the formula )
    - alpha, array of error in y, in the same units as y
    - m, the number of fitted parameters.
    Outputs:
    - chi2, the unreduced chi squared value
    """
    n = len(y) # number of observations/datapoints
    dof = n-m # degree of freedom, m is fitted params
    chi2 = chi_squared(y,yx,alpha) # get the unreduced chi squared
    reduced_chi2 = chi2/dof # reduce it
    return reduced_chi2

In [ ]:
# TODO stop using globals. Consider OO principles, or structured/recorded arrays.
# TODO error handling and exceptions need to be added, because this too is currently a mess.

def  add_own_data(filename):
    """Function to add archive data and your own data. Put it into three columns: tmid, error, and type of JD (Either
     "HJD" or "BJD", assuming HJD_UTC or BJD_TDB. Will be asked for the P_0: supply your own or use ETDs. For t_0,
     assumes earliest tmid given. NOTE: if ETD data has been used, it will force the ETD values.
    Inputs:
    - filename, the name/path of your data table (just use a csv)
    Outputs:
    - bjd, array of BJD_TDB mid-transit-times
    - mid_err, the uncertainty of those tmids
    - epoch, transits since t_0
    - t_0, midtime at epoch=0
    - p_0, the period at epoch=0
    """
    # TODO don't do this
    global url
    global ra
    global dec

    # import all the data
    mid,mid_err,jd_type = np.genfromtxt(filename, encoding='unicode_escape', delimiter=',', missing_values='',
                                            filling_values=-1, unpack=True)

    bjd=[]
    # check if we need to convert to BJD
    # TODO: error handling needs implementing here
    for i,tmid in enumerate(mid):
        if jd_type[i] == "BJD":
            bjd.append(tmid)
        elif jd_type[i] == "HJD":
            bjd.append(plc.hjd_utc_to_bjd_tdb(ra, dec, tmid))
        else:
            print("Something has gone wrong! Your type value needs to be a string of BJD or HJD.")
            # TODO: better way to deal with this, proper exception handling
            break

    # check if we need to assume a t_0 and p_0, if so, do so
    t_0 = 0
    p_0 = 0
    if not etd_used:
        # use the earliest tmid as t_0
        t_0 = np.amin(bjd)

        # ask for p_0, check it's sensible
        p_good = False
        while not p_good:
            p_0 = input("Please input the value of P, in days:")
            if p_0 > 0:
                p_good = True
                # TODO: a better check than this!
    else:
        # url should be defined already outside this function, so can just call it because we're bad people that
        # aren't obeying proper OO principles
        # TODO: obey proper OO principles
        p_0, t_0 = p0_t0_from_etd(url)

    epoch=[]

    # calculate the epoch system - assume closest rounding is correct via np.rint
    for i,tmid in enumerate(bjd):
        epoch[i] = np.rint((tmid-t_0)/p_0)

    #return everything: again, three arrays, two floats
    return bjd, mid_err, epoch, p_0, t_0

In [ ]:
def sort_etd_own_data(etd_data, own_data):
    """Sorts the filtered ETD data and user's archive data into the correct order.
    Inputs:
    -etd_data, an array of ETD data
    -own_data, archive data added from literature, may contain user's own datapoint
    Outputs:
    -sorted_data, the sorted combination of these two datasets
    """
    own_bjd, own_mid_err, own_epoch = own_data
    etd_bjd, etd_mid_err, etd_epoch = etd_data
    for j in tqdm(range(0, len(own_epoch))):
        for i in range(0, len(etd_epoch)):
            if (own_epoch[j] >= etd_epoch[i]):
                etd_epoch = np.insert(etd_epoch,i,own_epoch[j])
                etd_bjd = np.insert(etd_bjd,i,own_bjd[j])
                etd_mid_err = np.insert(etd_mid_err,i,own_mid_err[j])
                break
    # TODO: inefficient, better to add to new array than constantly cycling ETD array?

    return [etd_epoch, etd_bjd, etd_mid_err]